In [3]:
#import packages
import pandas as pd
from datetime import datetime

In [8]:
#Assign Global Mobility Report to url
url = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"

In [24]:
cols = [
        "country_region",
        "sub_region_1",
        "sub_region_2",
        "metro_area",
        "iso_3166_2_code",
        "census_fips_code",
        "date",
        "retail_and_recreation_percent_change_from_baseline",
        "grocery_and_pharmacy_percent_change_from_baseline",
        "parks_percent_change_from_baseline",
        "transit_stations_percent_change_from_baseline",
        "workplaces_percent_change_from_baseline",
        "residential_percent_change_from_baseline"
]

In [25]:
df = pd.read_csv(url, usecols=cols, low_memory=False)

In [26]:
#Convert date column to days of the year
df['date'] = pd.to_datetime(df['date'], format="%Y/%m/%d", utc=True)
df['date'] = df['date'].dt.dayofyear

In [27]:
#Standardise country names to OWID country names
# country_mapping = pd.read_csv("mobility_to_owid_countries.csv")
country_mapping = pd.read_csv("../input/owid/eu_countries.csv")
country_mapping.columns = ['country_region' , 'region']
df = country_mapping.merge(df, on="country_region")

In [28]:
df.head()

,country_region,region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,Austria,European Union,NaN,NaN,NaN,NaN,NaN,46,9.0,1.0,42.0,13.0,0.0,-2.0
1,Austria,European Union,NaN,NaN,NaN,NaN,NaN,47,15.0,21.0,42.0,12.0,1.0,-2.0
2,Austria,European Union,NaN,NaN,NaN,NaN,NaN,48,9.0,5.0,35.0,3.0,-4.0,0.0
3,Austria,European Union,NaN,NaN,NaN,NaN,NaN,49,8.0,5.0,40.0,2.0,-4.0,0.0
4,Austria,European Union,NaN,NaN,NaN,NaN,NaN,50,4.0,2.0,10.0,-1.0,-5.0,1.0


In [23]:
country_mapping.columns = ['Country' , 'country_region']
country_mapping.head()

,Country,country_region
0,Austria,European Union
1,Belgium,European Union
2,Bulgaria,European Union
3,Croatia,European Union
4,Cyprus,European Union


In [29]:
#Remove subnational data, keeping only country figures
filter_cols = ["sub_region_1", "sub_region_2","metro_area","iso_3166_2_code","census_fips_code"]
country_mobility = df[df[filter_cols].isna().all(1)]

In [30]:
#Delete columns
country_mobility = country_mobility.drop(columns=[ "sub_region_1", "sub_region_2","metro_area","census_fips_code","iso_3166_2_code"])

In [31]:
#Assign new column names
rename_dict = {
        "date": "Year",
        "retail_and_recreation_percent_change_from_baseline": "Retail & Recreation",
        "grocery_and_pharmacy_percent_change_from_baseline": "Grocery & Pharmacy",
        "parks_percent_change_from_baseline": "Parks",
        "transit_stations_percent_change_from_baseline": "Transit Stations",
        "workplaces_percent_change_from_baseline": "Workplaces",
        "residential_percent_change_from_baseline": "Residential"
    }

In [32]:
#Rename columns
country_mobility = country_mobility.rename(columns=rename_dict)

In [35]:
country_mobility.head()

,country_region,region,Year,Retail & Recreation,Grocery & Pharmacy,Parks,Transit Stations,Workplaces,Residential
0,Austria,European Union,46,9.0,1.0,42.0,13.0,0.0,-2.0
1,Austria,European Union,47,15.0,21.0,42.0,12.0,1.0,-2.0
2,Austria,European Union,48,9.0,5.0,35.0,3.0,-4.0,0.0
3,Austria,European Union,49,8.0,5.0,40.0,2.0,-4.0,0.0
4,Austria,European Union,50,4.0,2.0,10.0,-1.0,-5.0,1.0


In [34]:
#Save to files
country_mobility.to_csv('Google Mobility Trends (2020).csv', index=False)